# Power price validation and analyses

Routines for price validation and analyses of *pommesdispatch* or *pommesinvest* model runs for historical years comprising

Retrospective:
* Comparison of price statistics
* Comparison of price patterns
* Comparison of negative price and scarcity price events
* Calculation of error metrics (MAE, RSME)

Prospective:
* Plotting of prices
* Calculation of metrics

## Package imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pommesevaluation.price_validation import (
    read_and_reshape_historical_prices, read_and_reshape_smard_prices, compare_or_show_price_distribution,
    draw_price_plot, draw_weekly_plot, draw_price_duration_plot,
    calculate_error_metrics)
from pommesevaluation.pommesinvest_routines import is_leap_year

plt.rcParams.update({'font.size': 12})

## Set parameters
Most importantly, specify whether to analyze a dispatch or investment model run by choosing
`mode = "dispatch"` resp. `mode = "investment"`.

Specify path and filenames where data is stored:
* Power price results from the model.
* Historical day-ahead prices (used for validation)

In [ ]:
# Choose the mode here
mode = "investment"

# Choose year to evaluate (for investment model >= 2020)
simulation_year = 2020

# Choose settings (defaults might apply!)
basic_path = "./model_results/"
path_results = {
    "dispatch": f"{basic_path}pommesdispatch/",
    "investment": f"{basic_path}pommesinvest/",
}
path_plots = "./plots/"
path_processed_data = "./data_out/"
path_historical_prices = "./data/prices/"

# Specific settings for dispatch
eeg_clusters_per_technology = 20

# Specific settings for investment
time_frame_in_years = 26
frequency = "1H"
dr_scenario = "none"
dr_scenarios = ["none", "5", "50", "95"]
sensitivities = ["pv_-50%", "pv_+50%", "prices_-50%", "prices_+50%", "consumption_-50%", "consumption_+50%"]
fuel_price_scenario = "NZE"
emissions_pathway = "long-term"
impose_investment_maxima = False

if impose_investment_maxima:
    annual_investment_limits = ""
else:
    annual_investment_limits = "_no_annual_limit"
if dr_scenario != "none":
    file_add_on = (
        f"_with_dr_{dr_scenario}_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}_power-prices"
    )
else:
    file_add_on = (
        f"_no_dr_50_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}_power-prices"
    )

# Actual model file name
model_file_name = {
    "dispatch": (
        f"dispatch_LP_start-{simulation_year}-01-01_364-days_simple_complete_"
        f"{eeg_clusters_per_technology}_res-clusters_power-prices.csv"
    ),
    "investment": (
        f"investment_LP_start-2020-01-01_{time_frame_in_years}"
        f"-years_simple_freq_{frequency}{file_add_on}.csv"
    )
}

if simulation_year < 2022:
    compare = True
    color = ["b", "r"]
    ylim = None
else:
    compare = False
    color = "b"
    ylim = [-600, 1000]

## Read in data
* Historical prices for years from 2017 to 2021 (simulate year used for validation)
* Model prices for the respective simulation year

In [ ]:
historical_prices_fnames = {
    2017:
    f"{path_historical_prices}auction_spot_prices_germany_austria_2017.csv",
    2018: [
        f"{path_historical_prices}auction_spot_prices_germany_austria_2018.csv",
        f"{path_historical_prices}auction_spot_prices_germany_luxembourg_2018.csv"
    ]
}

for year in range(2019, 2021):
    historical_prices_fnames[year] = f"{path_historical_prices}auction_spot_prices_germany_luxembourg_{year}.csv"

for year in range(2021, 2024):
    historical_prices_fnames[year] = f"{path_historical_prices}smard_grosshandelspreise_stunde_2021-2023.xlsx"

historical_prices = dict()

for year, file_name in historical_prices_fnames.items():
    if year < 2021:
        historical_prices[year] = read_and_reshape_historical_prices(
            year, file_name
        )
    else:
        historical_prices[year] = read_and_reshape_smard_prices(
            year, file_name
        )

In [ ]:
model_prices_full = pd.read_csv(
    path_results[mode] + model_file_name[mode],
    sep=",",
    decimal=".",
    index_col=0,
    infer_datetime_format=True
)
model_prices_full.index = pd.to_datetime(model_prices_full.index)
model_prices = model_prices_full.loc[f"{simulation_year}"]
model_prices = model_prices.rename(columns={"Power price": "model_price"})

## Visualize historical prices and calculate some statistics
* Plot power prices for years 2017 to 2021
* Calculate summary statistics for all years

In [ ]:
loc_map = {
    2017: (0, 0),
    2018: (0, 1),
    2019: (0, 2),
    2020: (1, 0),
    2021: (1, 1),
    2022: (1, 2),
    2023: (2, 0),
}

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(25, 15), sharey=True)

for key, val in historical_prices.items():
    val.plot(ax=axes[loc_map[key]])
    axes[loc_map[key]].set_title(key)

axes[0, 0].set_ylabel("power price in €/MWh")
axes[1, 0].set_ylabel("power price in €/MWh")
axes[2, 0].set_ylabel("power price in €/MWh")
plt.tight_layout()
plt.show()

In [ ]:
stats_df = pd.DataFrame(index=historical_prices[2017].describe().index,
                        columns=range(2017, 2022))
stats_df.loc["negative_hours"] = 0
stats_df.loc["scarcity_hours"] = 0

for key, val in historical_prices.items():
    stats_df[key] = val.describe()
    stats_df.at["negative_hours",
                key] = val.loc[val["historical_price"] < 0].count()
    stats_df.at["scarcity_hours",
                key] = val.loc[val["historical_price"] == 3000].count()

stats_df

# Validate model results or show model results in isolation
Model results for a historical year are evaluated against historical day-ahead prices for past years.<br>
For future years, model results are displayed in isolation since there is no reference to compare against.
* Summary statistics are calculated for comparison.
* Negative price distributions are compared against each other.
* The annual price time series and duration curves are plotted against each other.
* Weekly price time series are written to disk.
* Mean average error (note: for evaluating estimates for the median) and root mean squared error (note: for evaluating estimates for the mean value) are evaluated.

## Compare summary statistics

In [ ]:
if simulation_year < 2022:
    if is_leap_year(simulation_year):
        historical_prices[simulation_year] = historical_prices[simulation_year].iloc[:8760]
    
    stats_comparison_df = pd.DataFrame(index=stats_df.index,
                                       columns=["historical", "model"])

    stats_comparison_df["historical"] = historical_prices[
        simulation_year].describe()
    stats_comparison_df["model"] = model_prices.describe()
    stats_comparison_df.at["negative_hours", "historical"] = (
        historical_prices[simulation_year].loc[
            historical_prices[simulation_year].historical_price < 0].count())
    stats_comparison_df.at["negative_hours", "model"] = model_prices.loc[
        model_prices.model_price < 0].count()
    stats_comparison_df.at["scarcity_hours", "historical"] = (
        historical_prices[simulation_year].loc[
            historical_prices[simulation_year].historical_price ==
            3000].count())
    stats_comparison_df.at["scarcity_hours", "model"] = model_prices.loc[
        model_prices.model_price == 3000].count()

else:
    stats_comparison_df = pd.DataFrame(index=stats_df.index, columns=["model"])

    stats_comparison_df["model"] = model_prices.describe()
    stats_comparison_df.at["negative_hours", "model"] = model_prices.loc[
        model_prices.model_price < 0].count()
    stats_comparison_df.at["scarcity_hours", "model"] = model_prices.loc[
        model_prices.model_price == 3000].count()

stats_comparison_df

## Compare resp. show negative price distribution

In [ ]:
if simulation_year > 2022:
    historical_prices = None
    negative_historical_prices = None
else:
    negative_historical_prices = historical_prices[simulation_year].loc[
        historical_prices[simulation_year].historical_price < 0
    ].historical_price

negative_model_prices = model_prices.loc[model_prices.model_price < 0].model_price
compare_or_show_price_distribution(
    negative_model_prices, 
    negative_historical_prices,     
    save=True,
    path_plots=path_plots,
    content="negative price distribution",
    file_name=f"negative_price_distribution_{simulation_year}", 
)

## Compare annual and weekly prices against each other

In [ ]:
if simulation_year < 2022:
    # Concatenate model results and historical ones
    prices = pd.concat([historical_prices[simulation_year], model_prices], axis=1)
else:
    prices = model_prices.copy()

draw_price_plot(
    power_prices=prices,
    color=color,
    title=f"Power price time series comparison for {simulation_year}",
    y_min_max=True,
    ylim=ylim,
    show=True,
    save=True,
    file_name=f"power_price_time_series_{simulation_year}",
    figsize=(15, 10),
)

In [ ]:
draw_weekly_plot(prices, simulation_year, ylim=ylim)

In [ ]:
if simulation_year < 2022:
    draw_price_duration_plot(
        model_prices,
        historical_prices[simulation_year],
        y_min_max=True,
        ylim=[-100, 200],
        show=True,
        save=True,
        file_name=f"power_price_duration_curve_{simulation_year}",
        figsize=(15, 10),
    )
else:
    model_prices_sorted = model_prices.sort_values(
    by="model_price", ascending=False).reset_index(drop=True)

    draw_price_plot(
        model_prices_sorted,
        color="b",
        title=f"Price duration curve for {simulation_year}",
        y_min_max=False,
        show=True,
        save=True,
        file_name=f"power_price_duration_curve_{simulation_year}"
    )

## Calculate error values
Calculate mean absolute and root mean squared error for model to get an impression on model results quality

In [ ]:
if simulation_year < 2022:
    error_metrics = calculate_error_metrics(historical_prices[simulation_year],
                                            model_prices)
    print(
        "Mean absolute error for model results: {:.2f} \nRoot mean squared error for model results: {:.2f}"
        .format(error_metrics["MAE"], error_metrics["RMSE"]))

In [ ]:
model_prices.min()

In [ ]:
model_prices.loc[f"{simulation_year}-04-30 13:00"]

In [ ]:
model_prices.idxmin()

## Evaluate spreads
Evaluate the following kind of "spreads":
* Single hour price differentials,
* Price differentials between hours that are up to 2, 4, 8, 12, 24 hours away from one another &rarr; evaluation for rolling windows and only for absolute spreads (not considering sign),
* For each of them, evaluate the maximum and minimum values.
* In addition, plot single hour spreads per year as well as their distribution.

In [ ]:
years = [y for y in model_prices_full.index.year.unique()]
spreads = pd.DataFrame(
    index=years,
    columns=["standard_deviation", "1H_min", "1H_max"] + [f"{hour_difference}H_abs_max" for hour_difference in [2, 4, 8, 12, 24]] + [f"{hour_difference}H_abs_min" for hour_difference in [2, 4, 8, 12, 24]]
)

for iter_year in years:
    model_prices_iter_year = model_prices_full.loc[model_prices_full.index.year == iter_year]
    spreads.at[iter_year, "standard_deviation"] = model_prices_iter_year.std().item()
    spreads.loc[iter_year, ["1H_min", "1H_max"]] = [model_prices_iter_year.diff().min().item(), model_prices_iter_year.diff().max().item()]
    for hour_difference in [2, 4, 8, 12, 24]:
        rolling_spreads = (
            model_prices_iter_year.rolling(hour_difference).max()
            - model_prices_iter_year.rolling(hour_difference).min()
        )
        spreads.at[iter_year, f"{hour_difference}H_abs_min"] = rolling_spreads.min().item()
        spreads.at[iter_year, f"{hour_difference}H_abs_max"] = rolling_spreads.max().item()
        
spreads.to_csv(
    f"{path_processed_data}/spreads_annual.csv",
    sep=";"
)

In [ ]:
for iter_year in years:
    fig, ax = plt.subplots(figsize=(15, 3))
    ax.set_title(f"Price spreads for {iter_year}")
    _ = model_prices_full.loc[model_prices_full.index.year == iter_year].diff().plot(ax=ax)
    _ = plt.tight_layout()
    plt.show()

In [ ]:
for iter_year in years:
    if iter_year <= 2022:
        hist_prices_diff = historical_prices[iter_year].diff()
    else:
        hist_prices_diff = None
    compare_or_show_price_distribution(
        model_prices_full.loc[model_prices_full.index.year == iter_year].diff(), 
        hist_prices_diff,
        bins=100,
        save=True,
        figsize=(15, 4),
        path_plots=path_plots,
        content=f"single hour price differentials for {iter_year}",
        file_name=f"single_hour_price_differentials_{iter_year}", 
    )